## Download libs

In [ ]:
!pip install requests beautifulsoup4 lxml

## Try to get all issue from the batch


In [ ]:
import requests

ISSUE = "issue-303"
BASE_URL = "https://www.deeplearning.ai"
JSON_URL = f"{BASE_URL}/_next/data/Ip2wXzUgHHESt2J9tyMBv/the-batch/{ISSUE}.json"

response = requests.get(JSON_URL)

if response.status_code == 200:
    data = response.json()
    post_data = data["pageProps"]['cmsData']['settings']

    print("🔵 Issue Title:", post_data["title"])
    print("🆔 Issue ID:", post_data["id"])

    # Статті лежать тут
    if "components" in post_data:
        print("\n📄 Articles:\n")
        for component in post_data["components"]:
            if component["type"] == "article":
                title = component.get("title", "No title")
                image_url = BASE_URL + component["image"]["url"] if "image" in component else "No image"
                html_content = component.get("html", "")

                print("📝 Title:", title)
                print("🖼️ Image:", image_url)
                print("📰 Preview (first 100 chars):", html_content[:100].replace('\n', ' '))
                print("—" * 50)
    else:
        print("No articles found in this issue.")
else:
    print("❌ Failed to fetch data:", response.status_code)

🔵 Issue Title: The Batch | DeepLearning.AI


KeyError: 'id'

In [ ]:
data["pageProps"]['cmsData']

{'settings': {'processEnv': {'siteUrl': 'https://www.deeplearning.ai',
   'platform': 'vercel',
   'darkMode': {'defaultMode': 'light', 'overrideOS': True},
   'nextImages': {'feature': True,
    'inline': False,
    'quality': 80,
    'source': False},
   'rssFeed': True,
   'memberSubscriptions': False,
   'commenting': {'system': None,
    'commentoUrl': 'https://cdn.commento.io',
    'disqusShortname': 'short-name-here'},
   'prism': {'enable': True, 'ignoreMissing': True},
   'toc': {'enable': False, 'maxDepth': 2},
   'isr': {'enable': False,
    'revalidate': 10,
    'maxNumberOfPosts': 20,
    'maxNumberOfPages': 20},
   'algoliaEnv': 'production'},
  'title': 'The Batch | DeepLearning.AI',
  'description': 'Weekly AI news for engineers, executives, and enthusiasts.',
  'logo': 'https://dl-staging-website.ghost.io/content/images/2021/04/LogoFiles_DeepLearning_PrimaryLogo.png',
  'icon': 'https://dl-staging-website.ghost.io/content/images/size/w256h256/2021/05/logo.png',
  'acce

In [ ]:
import requests
import json
import time

VERSION_HASH = "Ip2wXzUgHHESt2J9tyMBv"
BASE_URL = "https://www.deeplearning.ai"
DATA_URL_TEMPLATE = BASE_URL + f"/_next/data/{VERSION_HASH}/the-batch/issue-{{}}.json"

def fetch_issue_json(issue_number):
    url = DATA_URL_TEMPLATE.format(issue_number)
    try:
        r = requests.get(url, timeout=300)
        if r.status_code == 200:
            return r.json()
        else:
            print(f"Issue {issue_number} not found (status {r.status_code})")
    except Exception as e:
        print(f"Error fetching issue {issue_number}: {e}")
    return None

def parse_articles(json_data):
    try:
        article = json_data["pageProps"]["cmsData"]['post']
        results = []

        results.append({
            "issue": article["slug"],
            "title": article["title"],
            "feature_image": article['feature_image'],
            "custom_excerpt": article['custom_excerpt'],
            "article_url": f"{BASE_URL}/the-batch/{article['slug']}/"
            })
        return results
    except Exception as e:
        print(f"Error parsing issue: {e}")
        return []

def fetch_all_issues(start=1, end=303, delay=0.5):
    all_articles = []
    for i in range(start, end + 1):
        print(f"Fetching issue-{i}...")
        data = fetch_issue_json(i)
        if data:
            articles = parse_articles(data)
            all_articles.extend(articles)
        time.sleep(delay)
    return all_articles

if __name__ == "__main__":
    articles = fetch_all_issues()

    with open("all_batch_articles.json", "w", encoding="utf-8") as f:
        json.dump(articles, f, indent=2, ensure_ascii=False)

    print(f"\n✅ Total articles fetched: {len(articles)}")


Fetching issue-1...
Fetching issue-2...
Fetching issue-3...
Fetching issue-4...
Fetching issue-5...
Fetching issue-6...
Fetching issue-7...
Fetching issue-8...
Fetching issue-9...
Fetching issue-10...
Fetching issue-11...
Fetching issue-12...
Fetching issue-13...
Fetching issue-14...
Fetching issue-15...
Fetching issue-16...
Fetching issue-17...
Fetching issue-18...
Fetching issue-19...
Fetching issue-20...
Fetching issue-21...
Fetching issue-22...
Fetching issue-23...
Fetching issue-24...
Fetching issue-25...
Fetching issue-26...
Fetching issue-27...
Fetching issue-28...
Fetching issue-29...
Fetching issue-30...
Fetching issue-31...
Fetching issue-32...
Fetching issue-33...
Fetching issue-34...
Fetching issue-35...
Fetching issue-36...
Fetching issue-37...
Fetching issue-38...
Fetching issue-39...
Fetching issue-40...
Fetching issue-41...
Fetching issue-42...
Fetching issue-43...
Fetching issue-44...
Fetching issue-45...
Fetching issue-46...
Fetching issue-47...
Fetching issue-48...
F

# Try get all posts from ghost api

In [ ]:
import requests
import json

def fetch_all_posts():
    base_url = "https://dl-staging-website.ghost.io/ghost/api/content/posts/"
    api_key = "a4b216e975091c63cc39c1ac98"
    limit = 3000
    page = 1
    all_urls = []

    while True:
        # Construct the API request URL
        url = f"{base_url}?key={api_key}&limit={limit}&page={page}"

        try:
            # Make the API request
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad status codes

            # Parse JSON response
            data = response.json()
            posts = data.get('posts', [])

            # Extract URLs from posts
            for post in posts:
                if 'url' in post:
                    all_urls.append(post['url'])

            # Check pagination metadata
            meta = data.get('meta', {}).get('pagination', {})
            total_pages = meta.get('pages', 1)
            current_page = meta.get('page', 1)

            # Print progress
            print(f"Fetched page {current_page} of {total_pages}, {len(posts)} posts")

            # Break if we've reached the last page
            if current_page >= total_pages:
                break

            # Increment page for the next iteration
            page += 1

        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {page}: {e}")
            break

    return all_urls

def save_urls_to_file(urls, filename="post_urls.txt"):
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            for url in urls:
                f.write(url + '\n')
        print(f"Saved {len(urls)} URLs to {filename}")
    except Exception as e:
        print(f"Error saving URLs to file: {e}")

### Count number of urls

In [ ]:

# Fetch all post URLs
post_urls = fetch_all_posts()

# Print the number of URLs found
print(f"Total URLs fetched: {len(post_urls)}")

# Save URLs to a file
save_urls_to_file(post_urls)

Fetched page 1 of 1, 2183 posts
Total URLs fetched: 2183
Saved 2183 URLs to post_urls.txt


Extract all slugs (article name) and save to file

In [ ]:
def extract_slugs(input_file="post_urls.txt", output_file="post_slugs.txt"):
    base_url = "https://dl-staging-website.ghost.io/"
    slugs = []

    try:
        # Read URLs from the input file
        with open(input_file, 'r', encoding='utf-8') as f:
            urls = f.readlines()

        # Process each URL to extract the slug
        for url in urls:
            url = url.strip()  # Remove trailing newlines
            if url.startswith(base_url):
                # Extract the slug (remove base URL and trailing slash)
                slug = url[len(base_url):-1] if url.endswith('/') else url[len(base_url):]
                slugs.append(slug)
            else:
                # Skip URLs that don't match the expected pattern
                print(f"Skipping invalid URL: {url}")

        # Save slugs to the output file
        with open(output_file, 'w', encoding='utf-8') as f:
            for slug in slugs:
                f.write(slug + '\n')

        print(f"Extracted {len(slugs)} slugs and saved to {output_file}")

    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
    except Exception as e:
        print(f"Error processing file: {e}")

if __name__ == "__main__":
    extract_slugs()

Extracted 2183 slugs and saved to post_slugs.txt


## Test if all url works

In [ ]:
import requests
import time
from requests.exceptions import RequestException

def check_urls(input_file="post_slugs.txt", output_file="url_check_results.txt", delay=1):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    results = {"working": [], "redirected": [], "failed": []}

    try:
        # Read URLs from the input file
        with open(input_file, 'r', encoding='utf-8') as f:
            urls = [url.strip() for url in f.readlines()]

        print(f"Checking {len(urls)} URLs...")

        # Check each URL
        for i, url in enumerate(urls, 1):
            try:
                url = 'https://www.deeplearning.ai/the-batch/' + url
                # Send HEAD request first (lighter than GET)
                response = requests.head(url, headers=headers, timeout=10, allow_redirects=True)

                # If HEAD is not allowed, try GET
                if response.status_code in [405, 403]:
                    response = requests.get(url, headers=headers, timeout=10, allow_redirects=True)

                status_code = response.status_code
                final_url = response.url if response.url != url else None

                if status_code == 200:
                    results["working"].append(url)
                elif 300 <= status_code < 400:
                    results["redirected"].append(f"{url} -> {final_url} (Status: {status_code})")
                else:
                    results["failed"].append(f"{url} (Status: {status_code})")

                print(f"[{i}/{len(urls)}] {url}: Status {status_code}{' -> ' + final_url if final_url else ''}")

            except RequestException as e:
                results["failed"].append(f"{url} (Error: {str(e)})")
                print(f"[{i}/{len(urls)}] {url}: Error - {str(e)}")

            # Rate limit to avoid overwhelming the server
            time.sleep(delay)

        # Save results to output file
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("=== Working URLs (Status 200) ===\n")
            for url in results["working"]:
                f.write(f"{url}\n")
            f.write(f"\nTotal Working: {len(results['working'])}\n\n")

            f.write("=== Redirected URLs (Status 3xx) ===\n")
            for url in results["redirected"]:
                f.write(f"{url}\n")
            f.write(f"\nTotal Redirected: {len(results['redirected'])}\n\n")

            f.write("=== Failed URLs (Non-200/3xx or Errors) ===\n")
            for url in results["failed"]:
                f.write(f"{url}\n")
            f.write(f"\nTotal Failed: {len(results['failed'])}\n")

        print(f"\nResults saved to {output_file}")
        print(f"Summary: {len(results['working'])} Working, {len(results['redirected'])} Redirected, {len(results['failed'])} Failed")

    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
    except Exception as e:
        print(f"Error processing file: {e}")


In [ ]:
check_urls()

Checking 2183 URLs...
[1/2183] https://www.deeplearning.ai/the-batch/columbia-university-researchers-show-how-to-trick-trusting-ai-agents-with-poisoned-links: Status 200 -> https://www.deeplearning.ai/the-batch/columbia-university-researchers-show-how-to-trick-trusting-ai-agents-with-poisoned-links/
[2/2183] https://www.deeplearning.ai/the-batch/the-international-energy-agency-examines-the-energy-costs-and-potential-savings-of-the-ai-boom: Status 200 -> https://www.deeplearning.ai/the-batch/the-international-energy-agency-examines-the-energy-costs-and-potential-savings-of-the-ai-boom/
[3/2183] https://www.deeplearning.ai/the-batch/duolingo-turns-to-ai-translation-to-expand-its-most-popular-courses-to-all-28-user-languages: Status 200 -> https://www.deeplearning.ai/the-batch/duolingo-turns-to-ai-translation-to-expand-its-most-popular-courses-to-all-28-user-languages/
[4/2183] https://www.deeplearning.ai/the-batch/deepseek-r1s-update-leads-all-open-models-and-brings-it-up-to-date-with-th

## Test bad urls but with bigger timeout

In [ ]:
from requests.exceptions import RequestException

def retry_failed_urls(input_file="url_check_results.txt", output_file="url_check_failed_results.txt", timeout=100, delay=1):
    """
    Retry failed URLs from url_check_results.txt with a larger timeout and save results.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    results = {"working": [], "redirected": [], "failed": []}
    failed_urls = []

    try:
        # Read and parse failed URLs from input file
        with open(input_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        # Extract URLs from "Failed URLs" section
        in_failed_section = False
        for line in lines:
            line = line.strip()
            if line == "=== Failed URLs (Non-200/3xx or Errors) ===":
                in_failed_section = True
            elif line.startswith("===="):
                in_failed_section = False
            elif in_failed_section and line and not line.startswith('Total'):
                # Extract URL (before any status or message)
                url = line.split(' ')[0].strip()
                failed_urls.append(url)

        if not failed_urls:
            print("No failed URLs found in input file.")
            return

        print(f"Retrying {len(failed_urls)} failed URLs with timeout={timeout} seconds...")

        # Retry each URL
        for i, url in enumerate(failed_urls, 1):
            try:
                # Send HEAD request
                response = requests.get(url, headers=headers, timeout=timeout, allow_redirects=True)

                # If HEAD fails, try GET
                if response.status_code in [405, 403]:
                    response = requests.get(url, headers=headers, timeout=timeout, allow_redirects=True)
                    print('problem2')

                status_code = response.status_code
                final_url = response.url if response.url != url else ''

                # Check if redirect resolves to 200
                is_valid_redirect = False
                if 300 <= status_code < 400:
                    try:
                        print('problem1')
                        final_response = requests.head(final_url, headers=headers, timeout=timeout / 2)
                        is_valid_redirect = final_response.status_code == 200
                    except RequestException:
                        pass
                print(status_code)
                if status_code == 200 or is_valid_redirect:
                    results["working"].append(f"{url} {final_url if final_url else ''}")
                    print(f"[{i}/{len(failed_urls)}] {url}: Status {status_code} {'-> ' + final_url if final_url else ''} (Working)")
                elif 300 <= status_code < 400:
                    results["redirected"].append(f"{url} -> {final_url} (Status: {status_code})")
                    print(f"[{i}/{len(failed_urls)}] {url}: Status {status_code} -> {final_url}")
                else:
                    print('problem1f')
                    results["failed"].append(f"{url} (Status: {status_code})")
                    print(f"[{i}/{len(failed_urls)}] {url}: Status {status_code}")

            except RequestException as e:
                results["failed"].append(f"{url} (Error: {str(e)})")
                print(f"[{i}/{len(failed_urls)}] {url}: Error - {str(e)}")

            # Rate limit
            time.sleep(delay)

        # Save results to output file
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("=== Working URLs (Status 200 or Valid Redirects) ===\n")
            for url in results["working"]:
                f.write(f"{url}\n")
            f.write(f"\nTotal Working: {len(results['working'])}\n\n")

            f.write("=== Redirected URLs (Status 3xx) ===\n")
            for url in results["redirected"]:
                f.write(f"{url}\n")
            f.write(f"\nTotal Redirected: {len(results['redirected'])}\n\n")

            f.write("=== Failed URLs ===\n")
            for url in results["failed"]:
                f.write(f"{url}\n")
            f.write(f"\nTotal Failed: {len(results['failed'])}\n")

        print(f"\nResults saved to {output_file}")
        print(f"Summary: {len(results['working'])} Working, {len(results['redirected'])} Redirected, {len(results['failed'])} Failed")

    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
    except Exception as e:
        print(f"Error processing file: {e}")

In [ ]:
retry_failed_urls()

Retrying 296 failed URLs with timeout=100 seconds...
200
[1/296] https://www.deeplearning.ai/the-batch/how-tree-search-improves-ai-agents-ability-to-browse-the-web-and-complete-tasks: Status 200 -> https://www.deeplearning.ai/the-batch/how-tree-search-improves-ai-agents-ability-to-browse-the-web-and-complete-tasks/ (Working)
200
[2/296] https://www.deeplearning.ai/the-batch/ai-price-wars-drive-costs-down-as-competition-heats-up: Status 200 -> https://www.deeplearning.ai/the-batch/ai-price-wars-drive-costs-down-as-competition-heats-up/ (Working)
200
[3/296] https://www.deeplearning.ai/the-batch/the-ai-tools-that-helped-define-2024-a-special-winter-holiday-issue-of-data-points: Status 200 -> https://www.deeplearning.ai/the-batch/the-ai-tools-that-helped-define-2024-a-special-winter-holiday-issue-of-data-points/ (Working)
200
[4/296] https://www.deeplearning.ai/the-batch/harvard-university-releases-giant-dataset-of-public-domain-books: Status 200 -> https://www.deeplearning.ai/the-batch/h

In [ ]:
with open('url_check_failed_results.txt', 'r', encoding='utf-8') as f:
            lines = f.readlines()

urls = []
in_failed_section = False

for line in lines:
    line = line.strip()
    if line == "=== Working URLs (Status 200 or Valid Redirects) ===":
        in_failed_section = True
    elif line.startswith("      "):
        in_failed_section = False
    elif in_failed_section and line and not line.startswith('Total'):
        url = line.split(' ')[0].strip()
        urls.append(url)

with open('fixed.txt', 'w', encoding='utf-8') as f:
    f.write("=== Working URLs (Status 200 or Valid Redirects) ===\n")
    for url in urls:
        f.write(f"{url}\n")

As result Get 2080 working article

In [ ]:
def extract_slugs(input_file="Good_pages.txt", output_file="result.txt"):
    base_url = "https://www.deeplearning.ai/the-batch/"
    slugs = []

    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            urls = f.readlines()

        for url in urls:
            url = url.strip()
            if url.startswith(base_url):
                slug = url[len(base_url):-1] if url.endswith('/') else url[len(base_url):]
                slugs.append(slug)
            else:
                print(f"Skipping invalid URL: {url}")

        # Save slugs to the output file
        with open(output_file, 'w', encoding='utf-8') as f:
            for slug in slugs:
                f.write(slug + '\n')

        print(f"Extracted {len(slugs)} slugs and saved to {output_file}")

    except FileNotFoundError:
        print(f"Error: Input file '{input_file}' not found.")
    except Exception as e:
        print(f"Error processing file: {e}")

if __name__ == "__main__":
    extract_slugs()

Extracted 2080 slugs and saved to result.txt


In [ ]:
with open('result.txt', 'r', encoding='utf-8') as f:
    slugs = f.readlines()

slugs = [line.rstrip('\n') for line in lines]
slugs

['columbia-university-researchers-show-how-to-trick-trusting-ai-agents-with-poisoned-links',
 'the-international-energy-agency-examines-the-energy-costs-and-potential-savings-of-the-ai-boom',
 'duolingo-turns-to-ai-translation-to-expand-its-most-popular-courses-to-all-28-user-languages',
 'deepseek-r1s-update-leads-all-open-models-and-brings-it-up-to-date-with-the-latest-from-google-and-openai',
 'how-ai-fund-is-building-ai-builders',
 'issue-304',
 'how-ai-models-can-encourage-bad-behavior',
 'deepseek-r1-regains-open-weights-crown',
 'issue-303',
 'cut-research-funding-weaken-the-nation',
 'study-shows-openais-gpt-4o-model-can-identify-verbatim-excerpts-from-paywalled-oreilly-books',
 'researchers-describe-training-methods-and-hardware-choices-for-deepseeks-v3-and-r1-models',
 'googles-new-ai-offerings-include-veo-3-video-generator-lightweight-gemma-3n-updates-to-gemini-pro-and-ultra-and-more',
 'anthropic-debuts-new-claude-4-sonnet-and-claude-4-opus-models-featuring-top-benchmarks-i

In [ ]:
VERSION_HASH = "TtaX0QEZ44ayZqMM2jinr"
BASE_URL = "https://www.deeplearning.ai"
DATA_URL_TEMPLATE = BASE_URL + f"/_next/data/{VERSION_HASH}/the-batch/{{}}.json"

In [ ]:
import requests

def fetch_page_json(slug):

    url = DATA_URL_TEMPLATE.format(slug)
    print(url)

    try:
        r = requests.get(url, timeout=100)
        if r.status_code == 200:
            return r.json()
        else:
            print(f"Slug {slug} not found (status {r.status_code})")
    except Exception as e:
        print(f"Error fetching issue {slug}: {e}")
    return None

In [ ]:
example = fetch_page_json(slug=slugs[0])

https://www.deeplearning.ai/_next/data/TtaX0QEZ44ayZqMM2jinr/the-batch/columbia-university-researchers-show-how-to-trick-trusting-ai-agents-with-poisoned-links.json


# Fetch data and Try to create embeddings

In [ ]:
def parse_articles(json_data):
    try:
        article = json_data["pageProps"]["cmsData"]["post"]
        results = []

        # Extract primary fields
        article_data = {
            "issue": article.get("slug"),
            "uuid": article.get("id"),
            "title": article.get("title"),
            "feature_image": article.get("feature_image"),
            "custom_excerpt": article.get("custom_excerpt"),
            "article_url": f"{BASE_URL}/the-batch/{article['slug']}/",
            "published_at": article.get("published_at"),
            "authors": [author["name"] for author in article.get("authors", [])],
            "tags": [tag["name"] for tag in article.get("tags", [])],
            "summary": article.get("custom_excerpt") or article.get("excerpt"),
            "media_type": ["text"] + (["image"] if article.get("feature_image") else []),
            "published_at": article['published_at'],
            "content": article['html'],
        }

        # Embeddable text (combining title and excerpt for better context)
        title = article_data["title"] or ""
        excerpt = article_data["custom_excerpt"] or ""
        article_data["embeddable_text"] = f"{title}. {excerpt}".strip()

        # Optional: Add key points if available in custom fields
        if "plaintext" in article:
            article_data["key_points"] = [
                point.strip() for point in article["plaintext"].split("•") if point.strip()
            ]

        # External links (e.g., arXiv)
        external_links = []
        for card in article.get("cards", []):
            if card.get("type") == "link" and "arxiv.org" in card.get("url", ""):
                external_links.append({
                    "type": "arXiv",
                    "url": card["url"],
                    "title": card.get("title", "External resource")
                })
        article_data["pdf_or_external_links"] = external_links

        # Images (including the main one)
        images = []
        if article.get("feature_image"):
            images.append({
                "url": article["feature_image"],
                "alt": f"Main image for article: {article['title']}"
            })
        article_data["images"] = images

        # Notes for multimodal processing (optional, you can generate heuristically)
        if images:
            article_data["multimodal_notes"] = (
                "Use the image together with the summary and key points for cross-modal retrieval."
            )

        results.append(article_data)
        return results

    except Exception as e:
        print(f"Error parsing issue: {e}")
        return []


In [ ]:
parse_articles(example)

[{'issue': 'columbia-university-researchers-show-how-to-trick-trusting-ai-agents-with-poisoned-links',
  'uuid': '6840c874f80c050001e4ee73',
  'title': 'Phishing for Agents: Columbia University researchers show how to trick trusting AI agents with poisoned links',
  'feature_image': 'https://dl-staging-website.ghost.io/content/images/2025/06/unnamed---2025-06-04T165354.442-1.png',
  'custom_excerpt': 'Researchers identified a simple way to mislead autonomous agents based on large language models.',
  'article_url': 'https://www.deeplearning.ai/the-batch/columbia-university-researchers-show-how-to-trick-trusting-ai-agents-with-poisoned-links/',
  'published_at': '2025-06-04T15:29:00.000-07:00',
  'authors': ['DeepLearning.AI'],
  'tags': ['Machine Learning Research',
   'Tech & Society',
   'AI Agents',
   'AI Safety',
   'Jun 4, 2025'],
  'summary': 'Researchers identified a simple way to mislead autonomous agents based on large language models.',
  'media_type': ['text', 'image'],
  '

In [ ]:
parse_articles(fetch_page_json(slug=slugs[10]))

https://www.deeplearning.ai/_next/data/TtaX0QEZ44ayZqMM2jinr/the-batch/study-shows-openais-gpt-4o-model-can-identify-verbatim-excerpts-from-paywalled-oreilly-books.json


[{'issue': 'study-shows-openais-gpt-4o-model-can-identify-verbatim-excerpts-from-paywalled-oreilly-books',
  'uuid': '68387e3a432a830001a47d12',
  'title': 'Did GPT-4o Train on O’Reilly Books?: Study shows OpenAI’s model can identify verbatim excerpts from paywalled books',
  'feature_image': 'https://dl-staging-website.ghost.io/content/images/2025/05/unnamed--99--1.png',
  'custom_excerpt': 'A study co-authored by tech-manual publisher Tim O’Reilly shows that OpenAI trained GPT-4o on parts of his company’s books that were not made freely available.',
  'article_url': 'https://www.deeplearning.ai/the-batch/study-shows-openais-gpt-4o-model-can-identify-verbatim-excerpts-from-paywalled-oreilly-books/',
  'published_at': '2025-05-28T08:34:00.000-07:00',
  'authors': ['DeepLearning.AI'],
  'tags': ['Tech & Society',
   'Business',
   'AI Regulations',
   'Responsible AI',
   'May 28, 2025'],
  'summary': 'A study co-authored by tech-manual publisher Tim O’Reilly shows that OpenAI trained G

In [ ]:
def save_article_json(article_data, slug):
    filename = f"{slug}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(article_data, f, ensure_ascii=False, indent=4)
    print(f"Saved {filename}")

In [ ]:
import json

all_articles = []

for slug in slugs:
    print(f"Processing slug: {slug}")
    json_data = fetch_page_json(slug)
    if not json_data:
        print(f"Skipping {slug}, no data fetched.")
        continue
    parsed_article = parse_articles(json_data)
    if parsed_article:
        all_articles.extend(parsed_article)  # flatten list
    else:
        print(f"No article parsed for {slug}")

# Create final JSON with top-level "documents" key
rag_json = {
    "documents": all_articles
}

# Save to file
with open("all_articles.json", "w", encoding="utf-8") as f:
    json.dump(rag_json, f, ensure_ascii=False, indent=4)

print(f"Saved {len(all_articles)} articles to all_articles.json")

Processing slug: columbia-university-researchers-show-how-to-trick-trusting-ai-agents-with-poisoned-links
https://www.deeplearning.ai/_next/data/TtaX0QEZ44ayZqMM2jinr/the-batch/columbia-university-researchers-show-how-to-trick-trusting-ai-agents-with-poisoned-links.json
Processing slug: the-international-energy-agency-examines-the-energy-costs-and-potential-savings-of-the-ai-boom
https://www.deeplearning.ai/_next/data/TtaX0QEZ44ayZqMM2jinr/the-batch/the-international-energy-agency-examines-the-energy-costs-and-potential-savings-of-the-ai-boom.json
Processing slug: duolingo-turns-to-ai-translation-to-expand-its-most-popular-courses-to-all-28-user-languages
https://www.deeplearning.ai/_next/data/TtaX0QEZ44ayZqMM2jinr/the-batch/duolingo-turns-to-ai-translation-to-expand-its-most-popular-courses-to-all-28-user-languages.json
Processing slug: deepseek-r1s-update-leads-all-open-models-and-brings-it-up-to-date-with-the-latest-from-google-and-openai
https://www.deeplearning.ai/_next/data/TtaX0

## Using openai for embedings

In [ ]:
import json
import openai
import numpy as np
import faiss

openai.api_key = "Your Open Ai Key"

EMBEDDING_MODEL = "text-embedding-3-small"

# Завантаження статей
with open("all_batch_articles.json", "r", encoding="utf-8") as f:
    articles = json.load(f)

# Підготовка текстів
texts = [f"{a['title']}\n{a['custom_excerpt']}" for a in articles]

# Отримання ембедінгів
embeddings = []
batch_size = 20

for i in range(0, len(texts), batch_size):
    batch = texts[i:i + batch_size]
    response = openai.embeddings.create(input=batch, model=EMBEDDING_MODEL)
    for item in response.data:
        embeddings.append(np.array(item.embedding, dtype=np.float32))

# Створення FAISS індексу
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Збереження індексу
faiss.write_index(index, "the_batch.index")

# Збереження метаінформації
with open("the_batch_metadata.json", "w", encoding="utf-8") as f:
    json.dump(articles, f, indent=2, ensure_ascii=False)

print("✅ Індекс і метадані збережено.")


✅ Індекс і метадані збережено.
